In [1]:
import tensorflow
from tqdm import tqdm

import keras

import random
import pathlib
import cytoolz
import numpy
from keras.models import Sequential, model_from_json
from keras.layers import LSTM, Dense, Embedding, Bidirectional
from keras.layers import TimeDistributed
from keras.optimizers import Adam
import thinc.extra.datasets
from spacy.compat import pickle
import spacy
import pandas as pd
from sklearn import preprocessing
import keras
from sklearn import model_selection
#import seaborn as sns
from sklearn import metrics
#import matplotlib.pyplot as plt

Using TensorFlow backend.
/cellar/users/btsui/anaconda3/envs/deep_nlp_cpu/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/cellar/users/btsui/anaconda3/envs/deep_nlp_cpu/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


In [2]:
%%time 
inS_dir='/cellar/users/btsui/Data/nrnb01_nobackup/METAMAP/allSRS.pickle'

#sra_dump_pickle_dir='/cellar/users/btsui/Data/SRA/DUMP/sra_dump.pickle'

srsS=pd.read_pickle(inS_dir)
srsS=pd.Series(data=srsS.values,index=pd.MultiIndex.from_arrays([srsS.index.get_level_values(0),
                                                            srsS.index.get_level_values(1).str.lower()]) )
technical_meta_data_df=pd.read_pickle(sra_dump_pickle_dir)


CPU times: user 1min 5s, sys: 9.52 s, total: 1min 14s
Wall time: 1min 15s


In [3]:
attrib_a=srsS.index.get_level_values(1)

#attrib_m=(attrib_a=='cell line')|(attrib_a=='cell_line')

attrib_m=attrib_a=='title'

attrib_m.sum()

srsS_sub=srsS[attrib_m].drop_duplicates()

In [43]:
len(srsS_sub)

1354824

In [44]:
srsS_sub[srsS_sub.str.contains(' ')]

ERS069057   title                             Streptococcus pneumoniae
SRS554825   title             Pathogen sample from Salmonella enterica
SRS1024493  title                   Leaf sample from Camellia oleifera
SRS568274   title                      Escherichia coli 2-316-03 S3 C1
ERS239563   title    Salmonella enterica subsp. enterica serovar Ty...
SRS902280   title    Salmonella enterica subsp I enterica serovar T...
SRS1219231  title    Salmonella enterica subsp I enterica serovar S...
SRS812378   title    Non-tumor DNA sample from Blood of a human fem...
SRS2264477  title    Sample from a human male participant in the db...
DRS014254   title                            Orthetrum japonicum 24-12
ERS991645   title                 Hepatitis C virus; IHTP_HCVss6265663
SRS531126   title    Non-tumor DNA sample from a human female parti...
SRS1668168  title                               Pseudomonas sp. NFPP10
SRS354076   title                 General Sample for Anopheles gambiae
SRS249

In [45]:
len(srsS_sub)

1354824

In [56]:
srsS_sub.sample(n=100)

SRS344625   title                          GAF embryo ChIP, replicate1
DRS012573   title    CCR5 targeting RGEN transfected, Human K562 Ce...
SRS1430944  title    Single-cell RNA-seq in EB+6hrDox(iNgn2.Isl1.Lh...
SRS1637259  title                                       1772075299_C08
SRS582832   title                                            miRNA-118
SRS1343199  title                                           IRRI_75618
SRS2249812  title                                        w1118_HD_f_r3
SRS955956   title                    Nicotiana tabacum Yun87,ROOT, LT3
SRS904727   title    Salmonella enterica subsp I enterica serovar I...
SRS1561129  title                               coChIP3-K27me3-K4me3_1
ERS1145365  title                                       10184.10110112
SRS364870   title                                      R1_L2_t5_35C_LD
SRS879951   title          Ticam2 LPS 4 hours (Ticam2..MGLibB_53819T4)
ERS1043288  title                     S. aureus CC59 isolate 2245N0059
ERS110